In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
%matplotlib inline
import pymc as pm
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.special import logsumexp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import arviz as az
import networkx as nx
import collections.abc
collections.Iterable = collections.abc.Iterable
from causalgraphicalmodels import CausalGraphicalModel
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')


%config InlineBackend.figure_format = 'retina'

plt.style.use(['seaborn-v0_8-darkgrid','seaborn-v0_8-colorblind'])

#### Code 9.1

In [ ]:
num_weeks = 100000
positions = np.zeros(num_weeks)
current = 10

for i in range(num_weeks):
    positions[i] = current 

    proposal = current + np.random.choice([-1,1], size =1)
    proposal = 1 if proposal > 10 else 10 if proposal < 1 else proposal

    prob_move = proposal / current 
    current = proposal if np.random.uniform(0, 1) < prob_move else current

#### Code 9.2

In [ ]:
plt.scatter(np.arange(1,100), positions[1:100])

#### Code 9.3

In [ ]:
counts, _ = np.histogram(positions, bins=10)
plt.bar(np.arange(1, 11), counts, width=0.1, color='blue')  
plt.xlabel('island')
plt.ylabel('number of weeks');

#### Code 9.4

In [ ]:
D = [1,10,100,1000]
for d in D:
    T = 1000
    Y = np.random.multivariate_normal(mean=np.zeros(d), cov=np.eye(d), size=T)
    Rd = np.sqrt(np.sum(Y**2, axis=1))  
    sns.kdeplot(Rd, color='blue');

#### Code 9.5

In [ ]:
# U needs to return net-log-probability
def U(q, x, y, a=0, b=1, k=0, d=1):
    muy = q[0]
    mux = q[1]
    U = (norm.logpdf(y, muy, 1).sum() + 
         norm.logpdf(x, mux, 1).sum() + 
         norm.logpdf(muy, a, b) + 
         norm.logpdf(mux, k, d))
    return -U

#### Code 9.6

In [ ]:
# gradient function
# need vector of partial derivatives of U with respect to vector q

def U_gradient(x, y, q, a=0, b=1, k=0, d=1):
    muy = q[0]
    mux = q[1]
    G1 = ((y - muy) / (1**2)).sum() + (a - muy) / (b**2)
    G2 = ((x - mux) / (1**2)).sum() + (k - mux) / (d**2)
    return -np.array([G1, G2])

np.random.seed(7)
y = np.random.normal(0,1,50)
x = np.random.normal(0,1,50)
x = scale.fit_transform(x.reshape(-1,1)).flatten()
y = scale.fit_transform(y.reshape(-1,1)).flatten()


#### Code 9.7

In [ ]:
Q = {}
Q['q'] = np.array([-0.1, 0.2])

pr = 0.3
step = 0.03
L = 11
n_samples = 4
path_col = (0, 0, 0, 0.5)

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlabel('mux')
ax.set_ylabel('muy')
ax.set_xlim(-pr, pr)
ax.set_ylim(-pr, pr)
ax.set_aspect('equal')

ax.scatter(Q['q'][0], Q['q'][1], marker='x', color='black', s=100, linewidths=2)

for i in range(1, n_samples + 1):
    Q = HMC2(U, U_gradient, step, L, Q['q']) # you'll need to run the next cell in order to load HMC2 to memory (not sure why it's laid out like this)
    
    if n_samples < 10:
        for j in range(L):
            K0 = np.sum(Q['ptraj'][j, :]**2) / 2
            ax.plot(Q['traj'][j:j+2, 0], Q['traj'][j:j+2, 1], 
                   color=path_col, linewidth=1+2*K0)
        
        ax.scatter(Q['traj'][:L+1, 0], Q['traj'][:L+1, 1], 
                  marker='o', color='white', s=35, zorder=2)
        
        arrow = FancyArrowPatch(
            (Q['traj'][L-1, 0], Q['traj'][L-1, 1]),
            (Q['traj'][L, 0], Q['traj'][L, 1]),
            arrowstyle='->', 
            mutation_scale=35,
            color='black',
            alpha=0.5,
            zorder=3
        )
        ax.add_patch(arrow)
        
        ax.text(Q['traj'][L, 0], Q['traj'][L, 1], str(i), 
               fontsize=8, ha='left', va='center', 
               bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.8))
    
    marker = 'o'
    facecolor = 'black' if Q['accept'] == 1 else 'none'
    edgecolor = 'red' if abs(Q['dH']) > 0.1 else 'black'
    
    ax.scatter(Q['traj'][L, 0], Q['traj'][L, 1], 
              marker=marker, 
              facecolors=facecolor,
              edgecolors=edgecolor,
              s=100, 
              linewidths=2,
              zorder=4)

plt.tight_layout()
plt.show()

#### Code 9.8 - 9.10

In [ ]:
def HMC2(U, grad_U, epsilon, L, current_q):
    q = current_q.copy()
    p = np.random.normal(0, 1, size=len(q))
    current_p = p.copy()
    
    p = p - epsilon * grad_U(x, y, q, a=0, b=0.5, k=0, d=0.5) / 2
    
    qtraj = np.full((L+1, len(q)), np.nan)
    ptraj = np.full((L+1, len(q)), np.nan)
    qtraj[0, :] = current_q
    ptraj[0, :] = p

    #### Code 9.9 ########
    # Alternate full steps for position and momentum
    for i in range(L):
        q = q + epsilon * p
        if i != L-1:
            p = p - epsilon * grad_U(x, y, q, a=0, b=0.5, k=0, d=0.5)
            ptraj[i+1, :] = p
    qtraj[i+1, :] = q


    #### Code 9.10 ########
    p = p - epsilon * grad_U(x, y, q, a=0, b=0.5, k=0, d=0.5) / 2
    ptraj[L, :] = p
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = -p
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U(current_q, x, y, a=0, b=0.5, k=0, d=0.5)
    current_K = np.sum(current_p**2) / 2
    proposed_U = U(q, x, y, a=0, b=0.5, k=0, d=0.5)
    proposed_K = np.sum(p**2) / 2

    # Accept or reject the state at end of trajectory, returning either
    # the position at the end of the trajectory or the initial position

    accept = 0
    if np.random.uniform() < np.exp(current_U - proposed_U + current_K - proposed_K):
        new_q = q
        accept = 1
    else:
        new_q = current_q

    return {
        'q': new_q,
        'traj': qtraj,
        'ptraj': ptraj,
        'accept': accept
    }

#### Code 9.11

In [ ]:
d = pd.read_csv('data/rugged.csv', sep=';')
d['log_gdp'] = np.log(d['rgdppc_2000']) 
dd = d[~d['rgdppc_2000'].isna()]
dd['log_gdp_std'] = dd.log_gdp / dd.log_gdp.mean()
dd['rugged_std'] = dd.rugged / dd.log_gdp.max()
dd['cid'] = np.where(dd.cont_africa==1, 0, 1) #reminder: theres no real need to do this since pymc is zero-indexed but alas


#### Code 9.12

In [ ]:
#this is a waste of time since we're already doing it but whatever...

with pm.Model() as m8_3:
    sigma = pm.Exponential('sigma', 1)
    a = pm.Normal('a',1,.1, shape = 2)
    b = pm.Normal('b',0,.3, shape = 2)
    mu = pm.Deterministic('mu',a[dd.cid] + b[dd.cid] * (dd.rugged_std - .215))
    log_gdp_std = pm.Normal('log_gdp_std', mu = mu, sigma=sigma, observed = dd.log_gdp_std)
    trace_m8_3 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})

pm.summary(trace_m8_3, var_names = ['a','b','sigma'])


#### Coe 9.13

In [ ]:
data_slim = dd[
                ['log_gdp_std',
                 'rugged_std',
                 'cid']]
data_slim.describe().T

#### Cdoe 9.14

In [ ]:
# yes this is the exact same as we've been using
with pm.Model() as m9_1:
    sigma = pm.Exponential('sigma', 1)
    a = pm.Normal('a',1,.1, shape = 2)
    b = pm.Normal('b',0,.3, shape = 2)
    mu = pm.Deterministic('mu',a[dd.cid] + b[dd.cid] * (dd.rugged_std - .215))
    log_gdp_std = pm.Normal('log_gdp_std', mu = mu, sigma=sigma, observed = dd.log_gdp_std)
    trace_m9_1 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})



#### Code 9.15

In [ ]:
pm.summary(trace_m8_3, var_names = ['a','b','sigma'])

#### Code 9.16

In [ ]:
# all you need to do here is change the cores parameter in trace_m9_1 from 1 to 4.  This is bugging fro some reason on my machine those so I wont run it.
with pm.Model() as m9_1:
    sigma = pm.Exponential('sigma', 1)
    a = pm.Normal('a',1,.1, shape = 2)
    b = pm.Normal('b',0,.3, shape = 2)
    mu = pm.Deterministic('mu',a[dd.cid] + b[dd.cid] * (dd.rugged_std - .215))
    log_gdp_std = pm.Normal('log_gdp_std', mu = mu, sigma=sigma, observed = dd.log_gdp_std)
    trace_m9_1 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})



#### Code 9.17

In [ ]:
#this automatically shows when you turn the progressbar in pm.Sample() to True
with pm.Model() as m9_1:
    sigma = pm.Exponential('sigma', 1)
    a = pm.Normal('a',1,.1, shape = 2)
    b = pm.Normal('b',0,.3, shape = 2)
    mu = pm.Deterministic('mu',a[dd.cid] + b[dd.cid] * (dd.rugged_std - .215))
    log_gdp_std = pm.Normal('log_gdp_std', mu = mu, sigma=sigma, observed = dd.log_gdp_std)
    trace_m9_1 = pm.sample(cores = 1, progressbar=True,  idata_kwargs={'log_likelihood': True})


#### Code 9.18

In [ ]:
pm.summary(trace_m9_1, var_names = ['a','b','sigma'])

#### Code 9.19

In [ ]:
az.plot_pair(trace_m9_1);

#### Code 9.20

In [ ]:
az.plot_trace(trace_m9_1);

#### Code 9.21

In [ ]:
az.plot_rank(trace_m9_1);

#### Code 9.22

In [ ]:
y = [-1,1]
np.random.seed(11)
with pm.Model() as m9_2:
    sigma = pm.Exponential('sigma',.0001)
    alpha = pm.Normal('alpha', 0, 1000)
    mu = pm.Deterministic('mu', alpha)
    y = pm.Normal('y', mu, sigma, observed = y)
    trace_m9_2 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})


#### Code 9.23

In [ ]:
pm.summary(trace_m9_2, var_names = ['alpha','sigma'])

#### Code 9.24

In [ ]:
y = [-1,1]
with pm.Model() as m9_3:
    sigma = pm.Exponential('sigma',1)
    alpha = pm.Normal('alpha', 1, 10)
    mu = pm.Deterministic('mu', alpha)
    y = pm.Normal('y', mu, sigma, observed = y)
    trace_m9_3 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})
pm.summary(trace_m9_3, var_names = ['alpha','sigma'])

#### Code 9.25

In [ ]:
np.random.seed(41)
y = np.random.normal(0,1,100)

#### Code 9.26

In [ ]:
with pm.Model() as m9_4:
    sigma = pm.Exponential('sigma',1)
    a1 = pm.Normal('a1', 0, 1000)
    a2 = pm.Normal('a2', 0, 1000)
    mu = pm.Deterministic('mu', a1 + a2)
    y = pm.Normal('y', mu, sigma, observed = y)
    trace_m9_4 = pm.sample(cores = 1, progressbar=True,  idata_kwargs={'log_likelihood': True})
pm.summary(trace_m9_4, var_names = ['alpha','sigma'])

#### Code 9.27

In [ ]:
with pm.Model() as m9_5:
    sigma = pm.Exponential('sigma',1)
    a1 = pm.Normal('a1', 0, 10)
    a2 = pm.Normal('a2', 0, 10)
    mu = pm.Deterministic('mu', a1 + a2)
    y = pm.Normal('y', mu, sigma, observed = y)
    trace_m9_5 = pm.sample(cores = 1, progressbar=False,  idata_kwargs={'log_likelihood': True})
pm.summary(trace_m9_5, var_names = ['alpha','sigma'])